In [1]:
import pandas as pd

from pathlib import Path
import fuzzymatcher
recon_batch = pd.read_csv('recon_batch_202408021028.csv')
recon_train = pd.read_csv('Recon_labeled_train.xlsx - Sheet1.csv')
recon_combined = pd.read_csv('recon_combined.xlsx - recon_batch_202408021028.xls.csv')


# UNDERSTANDING THE DATA


## COLUMNS


### 1. ID
Batch id

### 2. Removed
True - deleted file
False - valid batch

### 3. Created
Date format : 
YYYY-MM-DD.
HH:MM:SS.mmm 
+0300: The timezone offset from UTC 

### 3. Updated
Date format : 
YYYY-MM-DD.
HH:MM:SS.mmm 
+0300: The timezone offset from UTC 

### 4. Acquisition method
api / upload

### 5. Kind
backend, fsp, others (3rd party)

### 6. Accuracy 
of what , recon maybe ?

### 7. Quantity
Number of records

### 8. Metadata
- varies from client to client but may contain :
- the ones tagged with * are standard

a) dateto: yyyy-mm-dd 
b) s3_uri3: bucket it is uploaded to / uploads / year = yyyy / month = mm / day = dd / client = 3 / filename 
bb) batch_id
c) datefrom: yyyy-mm-dd
d) filename: .csv
e) requestedId: a-b-c-d-e
f) account_id: account of thr client being updated
ff) upload_tag:
ff1) is_reupload:
ff2) is_reversal:
ff3) raw_num_rows:
ff4) clean_num_rows:
ff5) staged_filename: s3bucket/staged instead of uploads
ff6) data_source_type :
fff) recon_date:
fff1) upload_tag:
fff2) account_name:
fff3) session_name:
fff4) opening_balance:
g) trans_date_format: 
h) value_date_format:

### 9. client_id
199	Kuda
2	BlackCopper
34	Credrails
3	NombaInc
100	Capricorn NG
137	CapriconTest
166	Carbon

### 10. updated_by_id
User that made the upload

### 11. status
XPC/RUIP/DPC/XP/DRP/DPS/DFPC/F/DP


XPC	- Mostly date format issues

RUIP - 3 error -file already ingested

DPC	4 error -file already ingested,Missing required columns

XP	2 errors - 404 errors,null

DRP - Null/blank

DPS - Null/blank

DFPC - Null/blank

F - Unknown errors, date errors

DP - 1 error -Missing colum/description
	
Abbreviations :
	
DATA_PROCESSING_NOT_STARTED = "DPS"	

DATA_PROCESSING = "DP"	

DATA_LOADING_TO_DATABASE = "DLD"
	
DATA_CHECKING_DUPLICATES = "DCD"
	
DATA_CHECKING_DUPLICATES_COMPLETE = "DCDC"
	
DATA_FILE_PROCESSING = "DFP"
	
DATA_FILE_PROCESSING_COMPLETE = "DFPC"
	
DATA_REPROCESSING = "DRP"
	
DATA_PROCESSING_COMPLETE = "DPC"
	
RECONCILIATION_NOT_STARTED = "RPS"	

RECONCILIATION_IN_PROGRESS = "RP"
	
RECONCILIATION_COMPLETED = "RPC"
	
RECONCILIATION_DISPUTED = "RD"
	
DATA_PROCESSING_FAILED = "F"
	
DATA_DELETION_IN_PROGRESS = "XP"
	
DATA_DELETION_COMPLETE = "XPC"
	
CALCULATING_ACCURACY_STATS = "CAS"
	
ACCURACY_STATS_UPDATED = "ASU"
	
REFERENCES_DELETION_IN_PROGRESS = "RDIP"
	
REFERENCES_UPDATING_IN_PROGRESS = "RUIP"
	
DATA_REMEDIATION = "DRMP"
	

### 12. status_message
Describes problem encountered.
If blank, it is an exception

### 13. duplicate_check
Is the file a duplicate

### 14. processing_count
Times file has been processed

### 15. completed
Time processing was finished

### 16. read_only
is the file read_only or not 

### 17. type of error
classification of the error.


Error classification from previous dashboard :

    - expected string or bytes like object
    - unexpected data
    - balances mismatch
    - missing required column
    - out of date range transactions
    - the statement has transactions that have overlapping months
    - the transactions date may have been parsed wrongly 
    - wrong date
    - truncation error
    - system error
    - id_range
    - expecting value
    - wrong date format
    - wrong file format
    - cannot set a dataframe with multiple columns to single column description
    - cannot convert float nan to integer 
    

Error classifications from recon_labeled_train:

1.Large File 

2.unknown error

3.Out of date range/overlapping dates

4.Date format error
  
5.Column missing error
 
6.Closing Balance/Opening balance mismatch
 
7.Config issue
 
8.Other error

9.File format issue

10.Truncation error

11.conversion issue
  
12.No error


['Large File' 'unknown error' 'Out of date range/overlapping dates'
 'Date format error' 'Column missing error'
 'Closing Balance/Opening balance mismatch' 'Config issue' 'Other error'
 'Conversion issue' 'File format issue' 'Truncation error'
 'conversion issue' 'No error']

12

In [10]:
 #Change the font of the 'status_message' and 'Type of Error' column to lowercase
recon_combined['status_message'] = recon_combined['status_message'].str.lower()
recon_combined['Type of Error'] = recon_combined['Type of Error'].str.lower()
recon_combined

,id,removed,created,updated,acquisition_method,kind,accuracy,quantity,metadata,client_id,updated_by_id,status,status_message,duplicate_checked,processing_count,completed,read_only,Type of Error
0,168,False,2022-10-17 10:46:54.837 +0300,2022-10-17 15:10:55.604 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-09-30"", ""s3_uri"": ""s3://prod-...",3,68.0,F,this file is too big. try splitting the file i...,False,1,NaN,False,large file
1,6668,True,2023-02-07 12:33:48.927 +0300,2023-04-28 16:12:04.478 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2022-11-12"", ""s3_uri"": ""s3://prod-...",3,102.0,DPS,NaN,False,1,NaN,False,unknown error
2,155697,True,2024-07-04 22:56:03.185 +0300,2024-07-05 08:51:11.800 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2024-06-15"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,the statement has transactions that have overl...,False,1,NaN,False,out of date range/overlapping dates
3,17492,True,2023-04-08 15:44:43.763 +0300,2023-04-11 12:04:40.237 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2023-04-05"", ""s3_uri"": ""s3://prod-...",3,135.0,F,time data '2023-03-21 18:18:19 utc' does not m...,False,1,NaN,False,date format error
4,3072,False,2022-12-30 18:28:36.819 +0300,2023-01-09 12:35:21.166 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-11-30"", ""s3_uri"": ""s3://prod-...",3,135.0,F,the file requires a column named 'timecreated'...,False,1,NaN,False,column missing error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,158852,False,2024-07-26 10:44:21.005 +0300,2024-07-26 10:46:43.569 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2024-07-16"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,balance mismatch: the calculated running balan...,False,1,NaN,False,NaN
1991,158860,False,2024-07-26 11:59:39.510 +0300,2024-07-26 11:59:51.424 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2024-07-25"", ""s3_uri"": ""s3://prod-...",100,334.0,F,config parsing error: the mapping function as ...,False,1,NaN,False,NaN
1992,115105,True,2023-10-09 11:48:49.128 +0300,2023-10-09 12:59:03.630 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2023-10-02"", ""s3_uri"": ""s3://prod-...",100,35.0,XPC,the mapping function as config provided doesn'...,False,1,NaN,False,NaN
1993,114985,True,2023-10-09 10:00:02.668 +0300,2023-10-09 11:12:29.560 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2023-10-08"", ""s3_uri"": ""s3://prod-...",100,1256.0,XPC,unconverted data remains when parsing with for...,False,1,NaN,False,NaN


In [11]:
# Step 1: Identify the rows with 'Type of Error' filled
error_mapping = recon_combined.dropna(subset=['Type of Error']).set_index('status_message')['Type of Error'].to_dict()

# Step 2: Apply this mapping to fill the 'Type of Error' column
recon_combined['Type of Error'] = recon_combined.apply(
    lambda row: error_mapping.get(row['status_message'], row['Type of Error']),
    axis=1
)

recon_combined


,id,removed,created,updated,acquisition_method,kind,accuracy,quantity,metadata,client_id,updated_by_id,status,status_message,duplicate_checked,processing_count,completed,read_only,Type of Error
0,168,False,2022-10-17 10:46:54.837 +0300,2022-10-17 15:10:55.604 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-09-30"", ""s3_uri"": ""s3://prod-...",3,68.0,F,this file is too big. try splitting the file i...,False,1,NaN,False,large file
1,6668,True,2023-02-07 12:33:48.927 +0300,2023-04-28 16:12:04.478 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2022-11-12"", ""s3_uri"": ""s3://prod-...",3,102.0,DPS,NaN,False,1,NaN,False,config issue
2,155697,True,2024-07-04 22:56:03.185 +0300,2024-07-05 08:51:11.800 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2024-06-15"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,the statement has transactions that have overl...,False,1,NaN,False,out of date range/overlapping dates
3,17492,True,2023-04-08 15:44:43.763 +0300,2023-04-11 12:04:40.237 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2023-04-05"", ""s3_uri"": ""s3://prod-...",3,135.0,F,time data '2023-03-21 18:18:19 utc' does not m...,False,1,NaN,False,date format error
4,3072,False,2022-12-30 18:28:36.819 +0300,2023-01-09 12:35:21.166 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-11-30"", ""s3_uri"": ""s3://prod-...",3,135.0,F,the file requires a column named 'timecreated'...,False,1,NaN,False,column missing error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,158852,False,2024-07-26 10:44:21.005 +0300,2024-07-26 10:46:43.569 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2024-07-16"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,balance mismatch: the calculated running balan...,False,1,NaN,False,NaN
1991,158860,False,2024-07-26 11:59:39.510 +0300,2024-07-26 11:59:51.424 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2024-07-25"", ""s3_uri"": ""s3://prod-...",100,334.0,F,config parsing error: the mapping function as ...,False,1,NaN,False,config issue
1992,115105,True,2023-10-09 11:48:49.128 +0300,2023-10-09 12:59:03.630 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2023-10-02"", ""s3_uri"": ""s3://prod-...",100,35.0,XPC,the mapping function as config provided doesn'...,False,1,NaN,False,NaN
1993,114985,True,2023-10-09 10:00:02.668 +0300,2023-10-09 11:12:29.560 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2023-10-08"", ""s3_uri"": ""s3://prod-...",100,1256.0,XPC,unconverted data remains when parsing with for...,False,1,NaN,False,NaN


### CONSISTENCY CHECK
# Function to check if all values in a series are the same
def check_consistency(group):
    return group.nunique() == 1

# Group by 'Type of error' and check the consistency of 'status_message'
consistency_check = recon_combined.groupby('Type of Error')['status_message'].apply(check_consistency)

# Print inconsistent groups
inconsistent_groups = consistency_check[consistency_check == False].index.tolist()
if inconsistent_groups:
    print(f"Inconsistent groups found in 'Type of error': {inconsistent_groups}")
    inconsistent_rows = recon_combined[recon_combined['Type of Error'].isin(inconsistent_groups)]
    print(inconsistent_rows)
else:
    print("All groups are consistent.")


In [95]:
recon_combined.columns

Index(['id', 'removed', 'created', 'updated', 'acquisition_method', 'kind',
       'accuracy', 'quantity', 'metadata', 'client_id', 'updated_by_id',
       'status', 'status_message', 'duplicate_checked', 'processing_count',
       'completed', 'read_only', 'Type of Error'],
      dtype='object')

In [12]:
# Count the frequency of every entry in 'status_message'
all_status_messages = recon_combined['status_message'].value_counts().reset_index()

# Rename the columns for clarity
all_status_messages.columns = ['status_message', 'frequency']

In [13]:
all_status_messages

,status_message,frequency
0,the mapping function as config provided doesn'...,88
1,config parsing error: the mapping function as ...,80
2,the file was already ingested,56
3,an error occurred (404) when calling the heado...,56
4,expected column 'response' cannot be found. en...,29
...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1
913,2023-08-06 13:01:20' in a column considered as...,1
914,the calculated running balance of 259340201 wi...,1
915,some records (e.g 2023-oct-06) have an older d...,1


In [14]:

# Step 1: Create a mapping from status_message to Type of Error
error_mapping = recon_combined.dropna(subset=['Type of Error']).drop_duplicates('status_message').set_index('status_message')['Type of Error'].to_dict()

# Step 2: Use this mapping to add the Type of Error column to all_status_messages
all_status_messages['Type of Error'] = all_status_messages['status_message'].map(error_mapping)

all_status_messages


,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,NaN
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,NaN
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,NaN
913,2023-08-06 13:01:20' in a column considered as...,1,NaN
914,the calculated running balance of 259340201 wi...,1,NaN
915,some records (e.g 2023-oct-06) have an older d...,1,NaN


## DATE FORMAT


685 missing without including value date errors
628 with

In [71]:
# Define the specific phrase
transaction_date_error = 'date does not match expected pattern. kindly update this value in the file and try again'
transaction_date_error2 = 'time data'
transaction_date_error3 = 'unconverted data remains'

# ensure the date columns and the expected date pattern is correct

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(transaction_date_error, na=False), 'Type of Error'] = 'date format error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(transaction_date_error2, na=False), 'Type of Error'] = 'date format error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(transaction_date_error3, na=False), 'Type of Error'] = 'date format error'



all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## BALANCE MISMATCH

In [72]:
# Define the specific phrase
balance_mismatch_error = 'balance mismatch'
balance_mismatch_error2 = 'the calculated running balance'


# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(balance_mismatch_error, na=False), 'Type of Error'] = 'closing balance/opening balance mismatch'
                                                                                                                                  
all_status_messages.loc[all_status_messages['status_message'].str.contains(balance_mismatch_error2, na=False), 'Type of Error'] = 'closing balance/opening balance mismatch'
                                                                                                                                  

all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## FILE SIZE


In [73]:
# Define the specific phrase
file_size_error = 'is more than 400mb'

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_size_error, na=False), 'Type of Error'] = 'large file'

all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## CONVERSION

In [120]:
# Define the specific phrase
conversion_error = 'could not convert'
conversion_error2 = 'cannot convert'
conversion_error3 = 'object has no attribute'
conversion_error4 = 'unsupported operand'

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(conversion_error, na=False), 'Type of Error'] = 'conversion issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(conversion_error2, na=False), 'Type of Error'] = 'conversion issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(conversion_error3, na=False), 'Type of Error'] = 'conversion issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(conversion_error4, na=False), 'Type of Error'] = 'conversion issue'

all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## MISSING COLUMN

In [75]:
# Define the specific phrase
missing_column_error = 'ensure this column exist'
missing_column_error2 = 'could not be found from index'
missing_column_error3 = 'no columns to parse from file'
missing_column_error4 = 'requires a column named'
missing_column_error5 = 'a column intended to have'
missing_column_error6 = 'a column named'
missing_column_error7 = 'not available in index'
missing_column_error8 = 'only headers detected. upload file with headers and data.'
missing_column_error9 = 'missing data cells'

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error2, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error3, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error4, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error5, na=False), 'Type of Error'] = 'column missing error'

all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error6, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error7, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error8, na=False), 'Type of Error'] = 'column missing error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(missing_column_error9, na=False), 'Type of Error'] = 'column missing error'

all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## OUT OF DATE RANGE / OVERLAPPING DATES

In [76]:
# Define the specific phrase
out_of_date_range_error = 'have a later date than'
out_of_date_range_error2 = 'have an older date than'
out_of_date_range_error3 = 'the recon date provided'
out_of_date_range_error4 = 'transactions that have overlapping months'
out_of_date_range_error5 = 'wrong recon date'
out_of_date_range_error6 = 'date may have been parsed wrongly'


# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error, na=False), 'Type of Error'] = 'out of date range/overlapping dates'
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error2, na=False), 'Type of Error'] = 'out of date range/overlapping dates'
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error3, na=False), 'Type of Error'] = 'out of date range/overlapping dates'
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error4, na=False), 'Type of Error'] = 'out of date range/overlapping dates'
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error5, na=False), 'Type of Error'] = 'out of date range/overlapping dates'
all_status_messages.loc[all_status_messages['status_message'].str.contains(out_of_date_range_error6, na=False), 'Type of Error'] = 'out of date range/overlapping dates'


all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## FILE FORMAT

In [89]:
# Define the specific phrase
file_format_error = 'convert the file to csv'
file_format_error2 = 'convert to csv and try again.'
file_format_error3 = 'extension is not supported'
file_format_error4 = 'not supported between instances of'
file_format_error5 = 'has low categorization rate due to unusable transaction identifiers in the csv file.'
file_format_error6 = 'convert to .csv and re-upload.'
file_format_error7 = 'file extension'
file_format_error8 = 'convert it to csv'
file_format_error9 = 'bad crc-32 for file '

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error2, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error3, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error4, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error5, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error6, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error7, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error8, na=False), 'Type of Error'] = 'file format issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(file_format_error9, na=False), 'Type of Error'] = 'file format issue'

all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## TRUNCATION ERROR


In [84]:
# Define the specific phrase
truncation_error = 'truncation error'
truncation_error2 = 'truncation issue'
truncation_error3 = 'truncation or e-notation has been found'
truncation_error4 = 'truncated values'
# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(truncation_error, na=False), 'Type of Error'] = 'truncation error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(truncation_error2, na=False), 'Type of Error'] = 'truncation error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(truncation_error3, na=False), 'Type of Error'] = 'truncation error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(truncation_error4, na=False), 'Type of Error'] = 'truncation error'
all_status_messages

,status_message,frequency,Type of Error
0,the mapping function as config provided doesn'...,88,config issue
1,config parsing error: the mapping function as ...,80,config issue
2,the file was already ingested,56,other error
3,an error occurred (404) when calling the heado...,56,system error
4,expected column 'response' cannot be found. en...,29,column missing error
...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error
913,2023-08-06 13:01:20' in a column considered as...,1,date format error
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates


## CONFIG ISSUE

In [104]:
# Define the specific phrase
config_error = 'mapping function as config'
config_error2 = 'incorrect ingestion configurations'

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(config_error, na=False), 'Type of Error'] = 'config issue'
all_status_messages.loc[all_status_messages['status_message'].str.contains(config_error2, na=False), 'Type of Error'] = 'config issue'

all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## UNKNOWN ERROR

In [105]:
# Define the specific phrase

unknown_error = 'an unknown error has occurred'
unknown_error2 = 'function not implemented'
unknown_error3 = 'unknown error'
unknown_error4 = 'investigating the issue'

# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(unknown_error, na=False), 'Type of Error'] = 'unknown error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(unknown_error2, na=False), 'Type of Error'] = 'unknown error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(unknown_error3, na=False), 'Type of Error'] = 'unknown error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(unknown_error4, na=False), 'Type of Error'] = 'unknown error'

all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## OTHER ERROR

In [123]:
# Define the specific phrase
other_error = 'file manually deleted'
other_error2 = 'wrong number of items passed'
other_error3 = 'list index out of range'
other_error4 = 'expecting value:'
other_error5 = 'arg must be a'
other_error6 = 'file was already ingested'
other_error7 = 'processing never completed'
other_error8 = 'your file has unexpected data before the line that have column headers. kindly remove the preceding lines first and then upload.'
other_error9 = 'file is not a zip file'
other_error10 = 'kindly regenerate another file for re-upload'
other_error11 = 'no module named'
other_error12 = 'file was not uploaded correctly'
other_error13 = 'expected string or bytes-like object'
other_error14 = 'cannot set a dataframe with multiple columns to the single column'
other_error15 = 'can only concatenate str'
other_error16 = 'is out of bounds'
other_error17 = 'object is not subscriptable'
other_error18 = 'is out-of-bounds'
# Scan through the 'status_message' column and assign 'date format error' to 'Type of Error' if the phrase is found
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error2, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error3, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error4, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error5, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error6, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error7, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error8, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error9, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error10, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error11, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error12, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error13, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error14, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error15, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error16, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error17, na=False), 'Type of Error'] = 'other error'
all_status_messages.loc[all_status_messages['status_message'].str.contains(other_error18, na=False), 'Type of Error'] = 'other error'


all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## SYSTEM ERROR

In [124]:
# 404 / 403
system_error = 'when calling the headobject operation'

all_status_messages.loc[all_status_messages['status_message'].str.contains(system_error, na=False), 'Type of Error'] = 'system error'
all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## WRONG ACCOUNT UPLOAD

In [125]:

wrong_account_error = ' file name is very different '
wrong_account_error2 = 'kindly upload it to the right account '
wrong_account_error3 = 'wrong account'
wrong_account_error4 = 'this is a backend statement. kindly upload it to the right account'

all_status_messages.loc[all_status_messages['status_message'].str.contains(wrong_account_error, na=False), 'Type of Error'] = 'wrong account upload'
all_status_messages.loc[all_status_messages['status_message'].str.contains(wrong_account_error2, na=False), 'Type of Error'] = 'wrong account upload'
all_status_messages.loc[all_status_messages['status_message'].str.contains(wrong_account_error3, na=False), 'Type of Error'] = 'wrong account upload'
all_status_messages.loc[all_status_messages['status_message'].str.contains(wrong_account_error4, na=False), 'Type of Error'] = 'wrong account upload'
all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## TOKENIZATION ERROR

In [126]:
# 404 / 403
tokenization_error = 'tokenizing'

all_status_messages.loc[all_status_messages['status_message'].str.contains(tokenization_error, na=False), 'Type of Error'] = 'tokenization error'
all_status_messages

,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## ADDING CLASSIFICATION COL

In [127]:
import numpy as np


def classify_message(row):
    if row['Type of Error'] == 'closing balance/opening balance mismatch':
        return 'Customer'
    elif row['Type of Error'] == 'column missing error':
        return 'Customer and System'
    elif row['Type of Error'] == 'config issue':
        return 'Customer'
    elif row['Type of Error'] == 'conversion issue':
        return 'Customer and System'
    elif row['Type of Error'] == 'date format error':
        return 'Customer and System'
    elif row['Type of Error'] == 'file format issue':
        return 'Customer and System'
    elif row['Type of Error'] == 'large file':
        return 'Customer'
    elif row['Type of Error'] == 'other error':
        return 'Customer'
    elif row['Type of Error'] == 'out of date range/overlapping dates':
        return 'Customer'
    elif row['Type of Error'] == 'system error':
        return 'System'
    elif row['Type of Error'] == 'truncation error':
        return 'Customer'
    elif row['Type of Error'] == 'unknown error':
        return 'Customer and System'
    elif row['Type of Error'] == 'wrong account upload':
        return 'Customer'
    elif row['Type of Error'] == 'tokenization error':
        return 'System'
    

all_status_messages['classification'] = all_status_messages.apply(classify_message, axis=1)
all_status_messages


,status_message,frequency,Type of Error,classification
0,the mapping function as config provided doesn'...,88,config issue,Customer
1,config parsing error: the mapping function as ...,80,config issue,Customer
2,the file was already ingested,56,other error,Customer
3,an error occurred (404) when calling the heado...,56,system error,System
4,expected column 'response' cannot be found. en...,29,column missing error,Customer and System
...,...,...,...,...
912,2023-09-14t23:32:22.466342: an an unknown erro...,1,unknown error,Customer and System
913,2023-08-06 13:01:20' in a column considered as...,1,date format error,Customer and System
914,the calculated running balance of 259340201 wi...,1,closing balance/opening balance mismatch,Customer
915,some records (e.g 2023-oct-06) have an older d...,1,out of date range/overlapping dates,Customer


## USE DICTIONARY TO MAP ERRORS TO RECORDS

In [128]:
recon_batch.columns

Index(['id', 'removed', 'created', 'updated', 'acquisition_method', 'kind',
       'accuracy', 'quantity', 'metadata', 'client_id', 'updated_by_id',
       'status', 'status_message', 'duplicate_checked', 'processing_count',
       'completed', 'read_only', 'Type of Error', 'classification'],
      dtype='object')

In [129]:
# Merge the two DataFrames on the 'status_message' column
Recon_full_labeled = recon_combined.merge(all_status_messages[['status_message', 'Type of Error', 'classification']],on='status_message', how='left')

# The 'left' join ensures that all rows from Recon_combined are kept, 
# and matching 'Type of Error' and 'classification' values are added from all_status_messages.
Recon_full_labeled

,id,removed,created,updated,acquisition_method,kind,accuracy,quantity,metadata,client_id,updated_by_id,status,status_message,duplicate_checked,processing_count,completed,read_only,Type of Error_x,Type of Error_y,classification
0,168,False,2022-10-17 10:46:54.837 +0300,2022-10-17 15:10:55.604 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-09-30"", ""s3_uri"": ""s3://prod-...",3,68.0,F,this file is too big. try splitting the file i...,False,1,NaN,False,large file,large file,Customer
1,6668,True,2023-02-07 12:33:48.927 +0300,2023-04-28 16:12:04.478 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2022-11-12"", ""s3_uri"": ""s3://prod-...",3,102.0,DPS,NaN,False,1,NaN,False,config issue,NaN,NaN
2,155697,True,2024-07-04 22:56:03.185 +0300,2024-07-05 08:51:11.800 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2024-06-15"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,the statement has transactions that have overl...,False,1,NaN,False,out of date range/overlapping dates,out of date range/overlapping dates,Customer
3,17492,True,2023-04-08 15:44:43.763 +0300,2023-04-11 12:04:40.237 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2023-04-05"", ""s3_uri"": ""s3://prod-...",3,135.0,F,time data '2023-03-21 18:18:19 utc' does not m...,False,1,NaN,False,date format error,date format error,Customer and System
4,3072,False,2022-12-30 18:28:36.819 +0300,2023-01-09 12:35:21.166 +0300,upload,backend,0.0,0.0,"{""dateTo"": ""2022-11-30"", ""s3_uri"": ""s3://prod-...",3,135.0,F,the file requires a column named 'timecreated'...,False,1,NaN,False,column missing error,column missing error,Customer and System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,158852,False,2024-07-26 10:44:21.005 +0300,2024-07-26 10:46:43.569 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2024-07-16"", ""s3_uri"": ""s3://prod-...",3,2280.0,F,balance mismatch: the calculated running balan...,False,1,NaN,False,NaN,closing balance/opening balance mismatch,Customer
1991,158860,False,2024-07-26 11:59:39.510 +0300,2024-07-26 11:59:51.424 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2024-07-25"", ""s3_uri"": ""s3://prod-...",100,334.0,F,config parsing error: the mapping function as ...,False,1,NaN,False,config issue,config issue,Customer
1992,115105,True,2023-10-09 11:48:49.128 +0300,2023-10-09 12:59:03.630 +0300,upload,others,0.0,0.0,"{""dateTo"": ""2023-10-02"", ""s3_uri"": ""s3://prod-...",100,35.0,XPC,the mapping function as config provided doesn'...,False,1,NaN,False,NaN,config issue,Customer
1993,114985,True,2023-10-09 10:00:02.668 +0300,2023-10-09 11:12:29.560 +0300,upload,fsp,0.0,0.0,"{""dateTo"": ""2023-10-08"", ""s3_uri"": ""s3://prod-...",100,1256.0,XPC,unconverted data remains when parsing with for...,False,1,NaN,False,NaN,date format error,Customer and System


Note: 
Some records have to be manually filled in spreadsheets including those that have no status message
Final doc : https://docs.google.com/spreadsheets/d/1S22TT3TtqmKGllBeXXQWO3B1RQcMHCfi/edit?gid=586504814#gid=586504814